This notebook is a Colab-friendly update to CaImAn's demo notebook (demo_pipeline_CNMFE.ipynb), and has been adapted for use at the 2023 SFN Satellite Workshop.

# CNMF-E demo pipeline: Intro
This notebook demonstrates how to use Caiman for processing 1p microendoscopic data. It shows how to use Caiman for the following steps:

![cnmfe pipeline full](https://raw.githubusercontent.com/EricThomson/image_sandbox/main/images/full_cnmfe_workflow.jpg)

1. Apply the NoRMCorre (nonrigid motion correction) algorithm for motion correction.
2. Apply the constrained nonnegative matrix factorization endoscopic (CNMF-E) source separation algorithm to extract an initial estimate of neuronal spatial footprint and calcium traces.
3. Apply quality control metrics to evaluate the initial estimates, and narrow down to the final set of estimates.

Some tools for visualization of movies and results are also included.

> This demo follows a similar pattern to the CNMF demo in `demo_pipeline.ipynb`. It includes less explanation except where there are important differences. If you want to get a more explanation-heavy picture of the fundamentals, we suggest starting with `demo_pipeline.ipynb`.

## Installation
This can take a few minutes we are installing Caiman from scratch.

In [ ]:
# Clone CaImAn from Github
!git clone -b colab https://github.com/EricThomson/CaImAn.git
%cd /content/CaImAn

# Install relevant packages
!pip install -r requirements.txt

# Install CaImAn package
!pip install -e .
!pip install mpld3

# Install CaImAn manager for relevant datasets
!python caimanmanager.py install --inplace

### Imports and general setup
We first need to import the Python libraries we will use in the rest of the notebook and tweak some general settings. Don't worry about these details now, we will explain the important things when they come up. Note in the following, we import `caiman` as `cm`, so when you see `cm` in the rest of the notebook, it just means you are using something from the Caiman package.  

In [ ]:
import bokeh.plotting as bpl
import cv2
import datetime
import glob
import holoviews as hv
import imageio
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import os
import psutil
from pathlib import Path

try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.summary_images import local_correlations_movie_offline
import psutil
from scipy.ndimage import center_of_mass
from IPython.display import display, clear_output
from IPython.display import HTML
from base64 import b64encode
import bokeh
import bokeh.plotting as bpl
from bokeh.io import output_notebook
import holoviews as hv
output_notebook()
hv.extension('bokeh')

%env HV_DOC_HTML=true

Continuing with our basic setup, we will set up a logger, and also set some environment variables in case that wasn't done already in your shell.

In [ ]:
# set up logging
logging.basicConfig(format="{asctime} - {levelname} - [{filename} {funcName}() {lineno}] - pid {process} - {message}",
                    filename=None,
                    level=logging.WARNING, style="{") #logging level can be DEBUG, INFO, WARNING, ERROR, CRITICAL

# set env variables
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"

# Select file(s) to be processed
Here, we analyze the data in `data_endoscope.tif`. The `download_demo` function will download the  file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file.

Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm.

In [ ]:
movie_path = download_demo('data_endoscope.tif')
print(f"Original movie for demo is in {movie_path}")

## Load and visualize raw data
Using colab workarounds for viewing raw data.

In [ ]:
# in colab  -- 40 sec runtime
# load original movie
# display movie
movie_orig = cm.load(movie_path)
display_movie = True
if display_movie:
    ds_ratio = 0.2
    moviehandle = movie_orig.resize(1, 1, ds_ratio)
    min_, max_ = np.percentile(moviehandle, 0.0001), np.percentile(moviehandle, 99.9999)
    moviehandle = np.array((moviehandle-min_)/(max_-min_)*255,dtype='uint8')
    imageio.mimwrite('/root/caiman_data/demo_movie.mp4', moviehandle, fps = 10,  quality=7)
    mp4 = open('/root/caiman_data/demo_movie.mp4','rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

# Set up a cluster
To enable parallel computing we will set up a local cluster. The resulting variable `cluster` contains the pool of processors (CPUs) that will be used in later steps. If you use `dview=cluster` in later steps, then parallel processing will be used. If you use `dview=None` then no parallel processing will be used. The `num_processors_to_use` variable determines how many CPU dores you will use (when set to `None` it goes to the default of one less than the number available).

Note for free colab we use the max number available (2)

In [ ]:
print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
num_processors_to_use = 2

Set up a cluster of processors. If one has already been set up (the `cluster` variable is already in your namespace), then that cluster will be closed and a new one created.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'cluster' in locals():  # 'locals' contains list of current local variables
    print('Closing previous cluster')
    cm.stop_server(dview=cluster)
print("Setting up new cluster")
_, cluster, n_processes = cm.cluster.setup_cluster(backend='multiprocessing',
                                                 n_processes=num_processors_to_use,
                                                 ignore_preexisting=False)
print(f"Successfully set up cluster with {n_processes} processes")

# Set up some parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this parameter object later on with settings for source extraction. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

Note here we are setting `pw_rigid` to `False` as our data seems to mainly contain large-scale translational motion. We can always redo this later if it turns out to be a mistake.

In [ ]:
# dataset dependent parameters
frate = 10                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': movie_path,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

parameters = params.CNMFParams(params_dict=mc_dict)

# Motion Correction
The background signal in micro-endoscopic data is very strong and makes motion correction challenging. As a first step the algorithm performs high pass spatial filtering with a Gaussian kernel to remove the bulk of the lower-frequency background activity and enhance spatial landmarks.

The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data for CNMF).

After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data, so no information is lost before source separation. The motion corrected files are saved in memory mapped format. If no motion correction is performed (i.e., `motion_correct` was set to `False`), then the file gets directly memory mapped.

The following also plots the discovered displacements in x- and y- in the movie.

In [ ]:
%%time
if motion_correct:
    # do motion correction rigid
    mot_correct = MotionCorrect(movie_path, dview=cluster, **parameters.get_group('motion'))
    mot_correct.motion_correct(save_movie=True)
    fname_mc = mot_correct.fname_tot_els if pw_rigid else mot_correct.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mot_correct.x_shifts_els)),
                                     np.max(np.abs(mot_correct.y_shifts_els)))).astype(int)
    else:
        bord_px = np.ceil(np.max(np.abs(mot_correct.shifts_rig))).astype(int)
        # Plot shifts
        plt.plot(mot_correct.shifts_rig)  # % plot rigid shifts
        plt.legend(['x shifts', 'y shifts'])
        plt.xlabel('frames')
        plt.ylabel('pixels')
        plt.gcf().set_size_inches(6,3)

    bord_px = 0 if border_nan == 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(movie_path, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

Compare original (left) and motion corrected movie (right).

You will probably notice they look pretty similar, as there wasn't much motion to begin with. You can see from the shift plot (plotted above) that the extracted shifts were all very small.

In [ ]:
# load original and motion-corrected movie
movie_orig = cm.load(movie_path)
movie_corrected = cm.load(mot_correct.mmap_file)
# display movie
display_movie = True
if display_movie:
    ds_ratio = 0.2
    moviehandle = cm.concatenate([movie_orig.resize(1, 1, ds_ratio) - mot_correct.min_mov*mot_correct.nonneg_movie,
                    movie_corrected.resize(1, 1, ds_ratio)], axis=2)
    min_, max_ = np.percentile(moviehandle, 0.0001), np.percentile(moviehandle, 99.999)
    moviehandle = np.array((moviehandle-min_)/(max_-min_)*255,dtype='uint8')

    # create and view mp4 for colap
    imageio.mimwrite('/root/caiman_data/motion_correct.mp4', moviehandle, fps = 30, quality=7)
    # not sure why this has to be unindented to work
    mp4 = open('/root/caiman_data/motion_correct.mp4','rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video width=800 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

## Load memory mapped file
Memory mapping is discussed in some detail in `demo_pipeline.ipynb`.

In [ ]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')
images.shape

# Parameter setting for CNMF-E
Everything is now set up to run source extraction with CNMFE. We will construct a new parameter dictionary and use this to modify the *existing* `parameters` object, using the `change_params()` method.

There are *two* main differences between the CNMF and CNMFE source separation algorithms. The first is the background model (this is discussed in the sidebar below on the Ring Model). The second difference is in how the models are initialized. This is addressed below when we go over setting corr/pnr thresholds for initialization, which we did not have to do for our 2p data.

We will explain the important differences in more detail below. For now, note that we have set `gnb` to `0`: this is effectively the flag telling Caiman to use CNMFE instead of CNMF.

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None for CNMFE
gSig = np.array([3, 3])  # expected half-width of neurons in pixels
gSiz = 2*gSig + 1     # half-width of bounding box created around neurons during initialization
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
tsub = 2            # downsampling factor in time for initialization, increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization, increase if you have memory problems
gnb = 0             # number of background components (rank) if positive, set to 0 for CNMFE
low_rank_background = None  # None leaves background of each patch intact (use True if gnb>0)
nb_patch = 0        # number of background components (rank) per patch (0 for CNMFE)
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background (increase to 2 if slow)
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

parameters.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # True for 1p
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

Initialize the model

In [ ]:
cnmfe_model = cnmf.CNMF(n_processes=n_processes,
                        dview=cluster,
                        params=parameters)

> <h2>CNMF-E: The background (ring) model</h2>
>
> <p>Background activity is very ill-behaved with 1p recordings: it often fluctuates locally and is much larger in magnitude than the neural signals we want to extract. In other words, the large-scale background model used for CNMF is not sufficient for most 1p data. Hence, Pengcheng Zhou and others came up with a localized model of background activity for CNMFE: CNMFE represents the background at each pixel as the weighted sum of activity from a circle (or ring) of pixels a certain distance from that pixel. The distance of this ring from the reference pixel is set by the <em>ring_size_factor</em> parameter. This more complex pixel-wise background model explains why CNMFE is computationally more expensive than CNMF, and also why it works better to mop up large-scale localized background noise to find the neurons in your 1p data.</p>
>
> <p> When you set <em>gnb</em> in the CNMF model (usually to 1 or 2), you are setting the number of global background components to use. The fact that you can get away with so few is testament to how well-behaved the background activity is in 2p recordings compared to 1p. When we set <em>gnb</em> to 0 in Caiman, this is a flag telling Caiman's back end to switch to the ring model of the background activity.</p>
>
> <p> For more details on CNMFE you can see the <a href="https://elifesciences.org/articles/28728">original paper</a> and the <a href="https://elifesciences.org/articles/38173">Caiman paper</a>.</p>


## Key parameters for CNMFE
The key parameters for CNMFE are slightly different than for CNMF, but with some overlap. As we'll see, because of the high levels of background activity, we can't initialize the same way as with CNMF. We have two new extremely important parameters directly related to initialization that come into play.

`rf` (int): *patch half-width*
> `rf`, which stands for 'receptive field', is the half width of patches in pixels. The patch width is `2*rf + 1`. `rf` should be *at least* 3-4 times larger than the observed neuron diameter. The larger the patch size, the less parallelization will be used by Caiman. If `rf` is set to `None`, then CNMFE will be run on the entire field of view.

`stride_cnmf (int)`: *patch overlap*
> `stride_cnmf` is the overlap between patches in pixels (the actual overlap is `stride_cnmf + 1`). This should be at least the diameter of a neuron. The larger the overlap, the greater the computational load, but the results will be more accurate when stitching together results from different patches. This param should probably have been called 'overlap' instead of 'stride'.

`gSig (int, int)`: *half-width of neurons*
> `gSig` is roughly the half-width of neurons in your movie in pixels (height, width). It is the standard deviation of the mean-centered Gaussian used to filter the movie before initialization for CNMFE. It is related to the `gSiz` parameter, which is the size (in pixels) of a bounding box created around each seed pixel during initilialization. You will usually set `gSiz` to between `2*gSig` and `4*gSig` for CNMFE.

`merge_thr (float)`: *merge threshold*
> If the correlation between two spatially overlapping components is above `merge_thr`, they will be merged into one component.

`min_corr` (float): *minimum correlation*
> Pixels from neurons tend to be correlated with their neighbors. For initialization we select for pixels above a minimum correlation `min_corr`.  We discuss this more below.

`min_pnr` (float): *minimum peak to noise ratio*
> Set a threshoild peak-to-noise ratio. Pixels from neurons tend to have a high signal-to-noise ratio. For initialization we select for pixels above a minimum peak-to-noise-ratio `min_pnr`. We discuss this more below.

As we did in `demo_pipeline.ipynb`, let's define a convenience function to get these key params for cnmfe so we can print them as we iteratively muck about in paramter space:

Some functions we will use:

In [ ]:
def key_params_cnmfe(cnmfe_model):
    """
    Convenience function to return critical parameters given CNMFE estimator object.
    Returns dict with values of rf, stride, gSig, gSiz, merge_threshold, min_corr, min_pnr

    Note:
    gSiz is included because it depends on gSig and you want to make sure to change it when you change gSig.
    These are not set in stone: tweak for your own needs!
    """
    rf = cnmfe_model.params.patch['rf']
    stride = cnmfe_model.params.patch['stride']
    gSig = cnmfe_model.params.init['gSig']
    gSiz = cnmfe_model.params.init['gSiz']
    merge_thr = cnmfe_model.params.merging['merge_thr']
    min_corr = cnmfe_model.params.init['min_corr']
    min_pnr = cnmfe_model.params.init['min_pnr']


    key_params = {'min_corr': min_corr,
                  'min_pnr': min_pnr,
                  'rf': rf,
                  'stride': stride,
                  'gSig': gSig,
                  'gSiz': gSiz,
                  'merge_thr': merge_thr}

    return key_params

def nb_inspect_correlation_pnr(corr, pnr, cmap='jet', num_bins=100):
    """
    inspect correlation and pnr images to infer the min_corr, min_pnr for cnmfe

    Args:
        corr: ndarray
            correlation image created with caiman.summary_images.correlation_pnr

        pnr: ndarray
            peak-to-noise image created with caiman.summary_images.correlation_pnr

        cmap: string
            colormap used for plotting corr and pnr images
            For valid colormaps see https://holoviews.org/user_guide/Colormaps.html

        num_bins: int
            number of bins to use for plotting histogram of corr/pnr values

    Returns:
        Holoviews plot layout (typically just plots in notebook)
    """
    import functools as fct
    hv_corr = hv.Image(corr,
                       vdims='corr',
                       label='correlation').opts(cmap=cmap)
    hv_pnr = hv.Image(pnr,
                      vdims='pnr',
                      label='pnr').opts(cmap=cmap)

    def hist(im, rx, ry, num_bins=num_bins):
        obj = im.select(x=rx, y=ry) if rx and ry else im
        return hv.operation.histogram(obj, num_bins=num_bins)

    str_corr = (hv.streams.RangeXY(source=hv_corr).rename(x_range='rx', y_range='ry'))
    str_pnr = (hv.streams.RangeXY(source=hv_pnr).rename(x_range='rx', y_range='ry'))

    hist_corr = hv.DynamicMap(
        fct.partial(hist, im=hv_corr), streams=[str_corr])

    hist_pnr = hv.DynamicMap(
        fct.partial(hist, im=hv_pnr), streams=[str_pnr])

    hv_layout = (hv_corr << hist_corr) + (hv_pnr << hist_pnr)

    return hv_layout


def get_rectangle_coords(im_dims,
                         stride,
                         overlap):
    """
    Extract rectangle (patch) coordinates: a helper function used by view_quilt().

    Given dimensions of summary image (rows x colums), stride between patches, and overlap
    between patches, returns row coordinates of the patches in patch_rows, and column
    coordinates patches in patch_cols. This is meant to be used by plot_patches().

    Args:
        im_dims: array-like
            dimension of image (num_rows, num_cols)
        stride: int
            stride between patches in pixels
        overlap: int
            overlap between patches in pixels

    Returns:
        patch_rows: ndarray
            num_patch_rows x 2 array, where row i contains onset and offset row pixels for patch row i
        patch_cols: ndarray
            num_patch_cols x 2 array, where row j contains onset and offset column pixels for patch column j

    Note:
        Currently assumes square patches so takes in a single number for stride/overlap.
    """
    patch_width = overlap + stride

    patch_onset_rows = np.array(list(range(0, im_dims[0] - patch_width, stride)) + [im_dims[0] - patch_width])
    patch_offset_rows = patch_onset_rows + patch_width
    patch_offset_rows[patch_offset_rows > im_dims[0]-1] = im_dims[0]-1
    patch_rows = np.column_stack((patch_onset_rows, patch_offset_rows))

    patch_onset_cols = np.array(list(range(0, im_dims[1] - patch_width, stride)) + [im_dims[1] - patch_width])
    patch_offset_cols = patch_onset_cols + patch_width
    patch_offset_cols[patch_offset_cols > im_dims[1]-1] = im_dims[1]-1
    patch_cols = np.column_stack((patch_onset_cols, patch_offset_cols))

    return patch_rows, patch_cols


def rect_draw(row_minmax,
              col_minmax,
              color='white',
              alpha=0.3,
              ax=None):
    """
    Draw a single transluscent rectangle on given axes object.

    Args:
        row_minmax: array-like
            [row_min, row_max] -- 2-elt int bounds for rect rows
        col_minmax: array-like
            [col_min, col_max] -- int bounds for rect cols
        color : string
            rectangle color, default 'yellow'
        alpha : float
            rectangle alpha (0. to 1., where 1 is opaque), default 0.3
        ax : pyplot.Axes object
            axes object upon which rectangle will be drawn, default None

    Returns:
        ax (Axes object)
        rect (Rectangle object)
    """
    from matplotlib.patches import Rectangle

    if ax is None:
        ax = plt.gca()

    box_origin = (col_minmax[0], row_minmax[0])
    box_height = row_minmax[1] - row_minmax[0]
    box_width = col_minmax[1] - col_minmax[0]

    rect = Rectangle(box_origin,
                     width=box_width,
                     height=box_height,
                     color=color,
                     alpha=alpha)
    ax.add_patch(rect)

    return ax, rect


def view_quilt(template_image,
               stride,
               overlap,
               color='white',
               alpha=0.2,
               vmin=None,
               vmax=None,
               figsize=(6.,6.),
               ax=None):
    """
    Plot patches on template image given stride and overlap parameters on template image.
    This can be useful for checking motion correction and cnmf spatial parameters.
    It ends up looking like a quilt pattern.

    Args:
        template_image: ndarray
            row x col summary image upon which to draw patches (e.g., correlation image)
        stride (int) stride between patches in pixels
        overlap (int) overlap between patches in pixels
        color: matplotlib color
            Any acceptable matplotlib color (r,g,b), string, etc., default 'white'
        alpha (float) : patch transparency (0. to 1.: higher is more opaque), default 0.2
        vmin (float) : vmin for plotting underlying template image, default None
        vmax (float) : vmax for plotting underlying template image, default None
        figsize (tuple) : fig size in inches (width, height), default (6.,6.)
        ax (pyplot.axes): axes object in case you want to add quilt to existing axes

    Returns:
        ax: pyplot.Axes object

    Example:
        # Uses cnm object (cnm) and correlation image (corr_image) as template:
        patch_width = 2*cnm.params.patch['rf'] + 1
        patch_overlap = cnm.params.patch['stride'] + 1
        patch_stride = patch_width - patch_overlap
        ax = view_quilt(corr_image, patch_stride, patch_overlap, vmin=0.0, vmax=0.6);
    """
    im_dims = template_image.shape
    patch_rows, patch_cols = get_rectangle_coords(im_dims, stride, overlap)

    if ax is None:
      f, ax = plt.subplots(figsize=figsize)

    ax.imshow(template_image, cmap='gray', vmin=vmin, vmax=vmax)
    for patch_row in patch_rows:
        for patch_col in patch_cols:
            ax, _ = rect_draw(patch_row,
                              patch_col,
                              color=color,
                              alpha=alpha,
                              ax=ax)

    return ax

## Inspect summary images and set parameters
### Correlation-pnr plot
For CNMFE, Caiman uses the correlation and peak-to-noise (PNR) ratio for initialization, which will both tend to be high in regions that contain neurons. Hence, we set a threshold for both quantitites to remove the low correlation/low pnr regions, and highlight the regions higher in both metrics, those regions most likely to contain neuronal activity.

First, we calculate the correlation and pnr maps of the raw motion corrected movie after filtering with a mean-centered Gaussian with standard deviation `gSig` (for more information, see the sidebar below). These calculation can be computationally and memory demanding for large datasets, so we subsample if there are many thousands of frames:

In [ ]:
print(gSig)
gsig_tmp = (3,3)
correlation_image, peak_to_noise_ratio = cm.summary_images.correlation_pnr(images[::max(T//1000, 1)], # subsample if needed
                                                                           gSig=gsig_tmp[0], # used for filter
                                                                           swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

<img src="https://raw.githubusercontent.com/EricThomson/image_sandbox/main/images/bokeh_menu.jpg" align="right" width="200"></img>
Using `nb_inspect_correlation_pnr()`, you can inspect the correlation and PNR images to find reasonable threshold values for `min_corr` and `min_pnr`. You can adjust the range of values displayed in the plots shown below by choosing the Y-box select tool (third button from the left -- highlighted in yellow in the accompanying image) and selecting the desired region in the histograms to the right of each image. You can also use the pan button (first button on the left) to zoom/adjust the axis limits in the histogram to make it easier to see the limits.

In [ ]:
hv.extension('bokeh')
nb_inspect_correlation_pnr(correlation_image, peak_to_noise_ratio)

We are looking for a couple of things in the above plot:

1. Did we filter with a `gSig` value small enough so that we aren't blending different neurons together? To see what it is like when this happens, set `gsig_tmp` to `(6,6)` and inspect the above plots.
2. More importantly, we want to find the threshold correlation and pnr values so that the *lower* threshold eliminates most of the noise and blood vessels from the plots, leaving behind as many of the neural pixels as possible. For this data it will be at a correlation value lower bound between 0.8 and 0.9, and and pnr lower bound somewhere between 10 and 20 (as with CNMF, there is no perfect value: it is often an iterative search, but keep in mind it is better to have false positives later than false negatives).

You can tweak the parameters in the following cell (included are some values that are reasonable):

In [ ]:
print(key_params_cnmfe(cnmfe_model))

gsig_new = gSig # unchanged
gsiz_new = gsig_new*2 + 1
min_corr_new  = 0.85
min_pnr_new = 12

cnmfe_model.params.change_params(params_dict={'gSig': gsig_new,
                                              'gSiz': gsiz_new,
                                              'min_corr': min_corr_new,
                                              'min_pnr': min_pnr_new});

print(key_params_cnmfe(cnmfe_model))

> <h2>CNMFE initialization: More on correlation and peak-to-noise-ratio</h2>

<img src="https://raw.githubusercontent.com/EricThomson/image_sandbox/main/images/mn_centered_gaussian.jpg" align="right" width="200"></img>

> How are correlation and peak-to-noise ratio actually calculated? First Caiman convolves the motion corrected movie with a <i>mean-centered Gaussian</i> (example to the right). The sigma of the Gaussian is <em>gSig</em>, and mean centering is turned on by setting <em>center_psf</em> to <em>True</em>. This mean centering creates a Gaussian with a positive peak in the middle of width <i>approximately</i> <em>gSig/2</em>, surrounded by a negative trench, and sets the outer edge to be zero. This preprocessing filter serves to highlight neuronal peaks and smooth away low-frequency background components.
>
> The function <em>correlation_pnr()</em> applies this mean-centered Gaussian to each frame of the motion corrected movie and returns the correlation image of that movie, as well as the peak-to-noise-ratio (PNR). The correlation image is the correlation of each pixel with its neighbors. The PNR is the ratio of the maximum magnitude at a pixel to the noise value at that pixel (it is a fast and rough measure of signal-to-noise). As mentioned above, both of these values tend to be higher in actual neurons, and the CNMFE initialization procedure is to set a threshold for both quantities, take their <i>product</i>, and use the peaks in this product map to find <i>seed pixels</i> for initialization of the CNMFE source separation algorithm.
>
> More details on the initialization procedure used here can be found in the <a href="https://elifesciences.org/articles/28728">CNMFE paper</a>, or just by exploring the code.         

### Quilt plot for spatial parameters
As discussed in `demo_pipeline.ipynb`, the other important paramters are those used for dividing the movie into patches for parallelization of the algorithm. The same processe is used for CNMFE. Namely, select `rf` and `stride` parameters so that many 3-4 neuron diameters can fit in each patch, and at least one neuron fits in the overlap region between patches. You can visualize the patches using the `view_quilt()` function:

In [ ]:
# calculate stride and overlap from parameters
cnmfe_patch_width = cnmfe_model.params.patch['rf']*2 + 1
cnmfe_patch_overlap = cnmfe_model.params.patch['stride'] + 1
cnmfe_patch_stride = cnmfe_patch_width - cnmfe_patch_overlap
print(f'Patch width: {cnmfe_patch_width} , Stride: {cnmfe_patch_stride}, Overlap: {cnmfe_patch_overlap}');

# plot the patches
fig, patch_ax = plt.subplots(figsize=(6,6))
patch_ax = view_quilt(correlation_image,
                      cnmfe_patch_stride,
                      cnmfe_patch_overlap,
                      vmin=np.percentile(np.ravel(correlation_image),50),
                      vmax=np.percentile(np.ravel(correlation_image),99.5),
                      color='yellow',
                      alpha=0.4,
                      ax=patch_ax);
patch_ax.set_title(f'CNMFE Patch Width {cnmfe_patch_width}, Overlap {cnmfe_patch_overlap}');
mpld3.plugins.connect(fig, mpld3.plugins.Zoom())
mpld3.display()

These patches and overlaps may seem a bit large, but that is ok: our main concern is that they not be too small. If you wanted to change them, you could use `change_params` as described above, and as dicussed in the CNMF notebook.

Now that we are happy with our parameters, let's run the algorithm.

## Run the CNMF-E algorithm
This can take a while, given the complexity of the model.

In [ ]:
%%time
cnmfe_model.fit(images);

# Component Evaluation
Source extraction typically produces many false positives. Our next step is quality control: separating the results into "good" and "bad" neurons using two different metrics (discussed in detail in the CNMF notebook):

- **Signal-to-noise ratio (SNR)**: a minimum SNR is set for the calcium transients (`min_SNR`).
- **Spatial correlation**:  a minimum correlation is set between the shape of each component and the frames in the movie when that component is active (`rval_thr`).

> Caiman does *not* use the CNN classifier to sort neurons based on shape for 1p data: the network was trained on 2p data. Hence, we set the `use_cnn` param to `False`.

Here we set the two parameters and run `evaluate_components()` to see which pass muster:


In [ ]:
min_SNR = 3            # SNR threshold
rval_thr = 0.85    # spatial correlation threshold

quality_params = {'min_SNR': min_SNR,
                  'rval_thr': rval_thr,
                  'use_cnn': False}
cnmfe_model.params.change_params(params_dict=quality_params)

cnmfe_model.estimates.evaluate_components(images, cnmfe_model.params, dview=cluster)

print('*****')
print(f"Total number of components: {len(cnmfe_model.estimates.C)}")
print(f"Number accepted: {len(cnmfe_model.estimates.idx_components)}")
print(f"Number rejected: {len(cnmfe_model.estimates.idx_components_bad)}")

# Visualize results

In [ ]:
#%% plot contour plots of accepted and rejected components
cnmfe_model.estimates.plot_contours_nb(img=correlation_image,
                                       idx=cnmfe_model.estimates.idx_components);

These components look reasonable, if a bit large -- their centers are reasonable but the spatial footprints are quite spread out. If I were persuing this further, it would likely be helpful to re-run CNMFE reducing `gSiz`, which can influence the overall spread of the neurons in space.

View traces of accepted and rejected components:

In [ ]:
cnmfe_model.estimates.nb_view_components(img=correlation_image,
                                        idx=cnmfe_model.estimates.idx_components,
                                        cmap='viridis', #gray
                                        thr=.9); #increase to see full footprint

In [ ]:
# rejected components
cnmfe_model.estimates.nb_view_components(img=correlation_image,
                                        idx=cnmfe_model.estimates.idx_components_bad,
                                        cmap='viridis', #gray
                                        denoised_color='red',
                                        thr=0.9); #increase to see full footprint

# A few loose ends
We have extracted the calcium traces C, spatial footprints A, and estimated spike counts S, which is the main goal with CNMF. But there are a few important things remaining.

##  Deconvolution for 1p?
While we haven't discussed deconvolution (the estimation of the spikes that generated the calcium traces in `C`), we suggest treating the spike counts returned for 1p data (in `estimates.S`) with CNMFE with some caution. Currently (as of Fall 2023) we are aware of no no ground-truth data that compares 1p recordings with actual spiking data. There is a *lot* of such data for 2p data, which allows for great comparison of different methods (for instance see [the Spikefinder paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006157)).

Because of this, most researchers analyze the calcium traces directly for 1p recordings (the data in `estimates.C`) or a normalized version of them.

## Extract $\Delta F/F$ values
Currently in Caiman, we don't return a true dfof value for 1p data because Caiman normalizes to both the baseline fluorescence and background activity, and the background activity in 1p is so ill-behaved (as discussed above in the sidebar on the ring model). This is likely to change soon, but we currently only *detrend* the data but do not normalize to baseline (which explains the warning you will see when you run the following):

In [ ]:
if cnmfe_model.estimates.F_dff is None:
    print('Calculating estimates.F_dff')
    cnmfe_model.estimates.detrend_df_f(quantileMin=8,
                                      frames_window=250,
                                      use_residuals=False);  # use denoised data
else:
    print("estimates.F_dff already defined")

## View some different movie results
As with CNMF, the CNMFE model of the original movie is:

    original_movie = neural_activity + background + residual
    
The main between CNMF and CNMFE is the model of the background. We can reconstruct the neural movie as `AC` just as we did in `demo_pipeline.ipynb`. Unfortunately, reconstructing the background activity via the ring model is much more complicated for CNMFE, so we will just punt to a built-in function for that in what follows (`compute_background()`).

Get model of neural activity and background activity (note for the neural model we are just including the accepted components):

In [ ]:
neural_activity = cnmfe_model.estimates.A[:, cnmfe_model.estimates.idx_components] @\
                  cnmfe_model.estimates.C[cnmfe_model.estimates.idx_components, :]  # AC
neural_movie = cm.movie(neural_activity).reshape(dims + (-1,), order='F').transpose([2, 0, 1])
background_model = cnmfe_model.estimates.compute_background(Yr);  # build in function -- explore source code for details
bg_movie = cm.movie(background_model).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

To view just the movie of pure neural activity:

In [ ]:
ds_ratio = 0.2
moviehandle = neural_movie.resize(1, 1, ds_ratio)
min_, max_ = np.percentile(moviehandle, 0.0001), np.percentile(moviehandle, 99.9999)
moviehandle = np.array((moviehandle-min_)/(max_-min_)*255,dtype='uint8')
imageio.mimwrite('/root/caiman_data/demo_movie.mp4', moviehandle, fps = 10,  quality=7)
mp4 = open('/root/caiman_data/demo_movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display(HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

Next we can view just the background model. You will see many regions that are constant such as blood vessels, but also lots of large-scale background flourescence, and some local activity which is is on spatial scales larger than `gSig`:

In [ ]:
ds_ratio = 0.2
moviehandle = bg_movie.resize(1, 1, ds_ratio)
min_, max_ = np.percentile(moviehandle, 0.0001), np.percentile(moviehandle, 99.9999)
moviehandle = np.array((moviehandle-min_)/(max_-min_)*255,dtype='uint8')
imageio.mimwrite('/root/caiman_data/demo_movie.mp4', moviehandle, fps = 10,  quality=7)
mp4 = open('/root/caiman_data/demo_movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display(HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

Note there are a **lot more** things in the main demo notebook at the github repo. For this limited demo we are just giving a flavor of how things will work when you run locally!

# Clean up open resources
We have a few resources we have left open we should take care of.

## Shut down cluster
To free up processing resources, let's shut down the cluster in case it is still open.

In [ ]:
cm.stop_server(dview=cluster)

## Shut down logger

In [ ]:
# Shut off logger
logging.shutdown()